# Example of a variational quantum circuit.

This is a very simple example of how to generate a custom circuit with a custom loss function.

In [1]:
from jqiskit.api import QuantumCircuit
import numpy as np

# Build a simple three bit quantum circuit.
n_qubits = 4
qc = QuantumCircuit(n_qubits)
qc.h(0)
qc.h(1)
qc.cx(0, 1)
# Some weird gates based on a phase shift gate.
qc.parametric('[[1., 0.,],[0., exp(1.0j * global_0)]]', 0)
qc.parametric('[[exp(-1.0j * global_1), 0.,],[0., exp(-1.0j * global_0)]]', 1)
qc.sqrtnot(0)
qc.sqrtnot(1)
qc.cswap(0, 1, 2)

print(qc)

Hadamard: (0,)
Hadamard: (1,)
CX: (0, 1)
Parametric: (0,)
Parametric: (1,)
SQRTNOT: (0,)
SQRTNOT: (1,)
CSWAP: (0, 1, 2)


In [2]:
def cost_function(global_0, global_1):
    """Simple cost function to put into the black-box optimizer."""
    counts, _ = qc.measure(num_shots=100, feed_dict={'global_0': global_0, 'global_1': global_1})
    values = list(counts.values())
    # I just made up some nonsense cost here.
    return (np.mean(values) / np.sum(values) - 0.25)**2

In [3]:
from scipy.optimize import minimize

x0 = [np.pi / 2, np.pi / 2]
minimize(lambda x_in: cost_function(x_in[0], x_in[1]),
         x0, method='powell',
         bounds=((0.0, 2*np.pi), (0.0, 2 * np.pi)),
         tol=1e-5)

   direc: array([[ 0.        ,  1.        ],
       [-1.57168623, -1.75872693]])
     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 139
     nit: 2
  status: 0
 success: True
       x: array([3.63730826, 3.8832105 ])